# Grand canonical ensemble transition-matrix Monte Carlo

In this example, [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst) methods are employed for a small [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst) range from 0 to 5 particles.
To begin, the system is initialized with the minimum number of particles by setting [Metropolis](../../monte_carlo/doc/Metropolis_arguments.rst) acceptance [Criteria](../../monte_carlo/doc/Criteria_arguments.rst) with favorable conditions for adding particles.
The [Metropolis](../../monte_carlo/doc/Metropolis_arguments.rst) criteria are then replaced with [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst).
Typical analysis from the previous tutorials are added, and we add [Checkpoint](../../utils/doc/Checkpoint_arguments.rst) files, [CriteriaUpdater](../../steppers/doc/CriteriaUpdater_arguments.rst), [CriteriaWriter](../../steppers/doc/CriteriaWriter_arguments.rst), and average [Energy](../../steppers/doc/Energy_arguments.rst) of a given [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst).
Finally, the simulation is run until the requested convergence criteria for the [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst) algorithm are [complete](../../monte_carlo/doc/Run_arguments.rst).

A small [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst) range allows the simulation to run quickly with ample sampling, and thus it is an ideal starting point to test the simulations. To begin, read the previous [NIST SRSW](https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website) values [for LJ](https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm) from file for comparison.

In [1]:
import pandas as pd
import numpy as np

srsw_file = "../test/data/stat150.csv"
fh=pd.read_csv(srsw_file)[:6]
fh['lnPI'] -= np.log(sum(np.exp(fh['lnPI'])))   # renormalize
fh['delta_ln_prob']=fh['lnPI'].diff()
fh

,N,energy,lnPI,energystd,lnPIstd,delta_ln_prob
0,0,-2.312265e-10,-18.707570,6.689238e-10,0.037092,NaN
1,1,-6.057402e-04,-14.037373,6.709198e-10,0.036838,4.670197
2,2,-3.057422e-02,-10.050312,9.649147e-06,0.036964,3.987061
3,3,-8.992832e-02,-6.458921,1.387472e-04,0.037746,3.591391
4,4,-1.784571e-01,-3.145637,3.315245e-05,0.038097,3.313283
5,5,-2.961920e-01,-0.045677,1.348791e-05,0.038458,3.099960


Run a short [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst) simulation to compare with these [NIST SRSW](https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm) values.

In [2]:
script="""
MonteCarlo
Configuration cubic_side_length 8 particle_type0 /feasst/particle/lj.txt
Potential Model LennardJones
Potential VisitModel LongRangeCorrections
ThermoParams beta {beta} chemical_potential0 -2.352321
FlatHistogram Macrostate MacrostateNumParticles width 1 min 0 max 5 \
              Bias TransitionMatrix min_sweeps 50
TrialTranslate weight 0.25 tunable_param 1
TrialTransfer weight 1 particle_type 0
Log trials_per_write 1e5 output_file lj.txt
Movie trials_per_write 1e5 output_file lj.xyz
CheckEnergy trials_per_update 1e5 tolerance 1e-8
Tune
CriteriaUpdater trials_per_update 1e5
CriteriaWriter trials_per_write 1e5 output_file lj_fh.txt
Energy output_file lj_en.txt trials_per_update 1 trials_per_write 1e5 multistate true
Checkpoint checkpoint_file checkpoint.fst num_hours 0.0001
Run until complete
""".format(beta=1./1.5)

def run(script):
    with open('script1.txt', 'w') as file: file.write(script)
    import subprocess
    syscode = subprocess.call("../../../build/bin/fst < script1.txt > script1.log", shell=True, executable='/bin/bash')
    with open('script1.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)
run(script)

# Usage: ./fst < file.txt
FEASST version 0.25.11
MonteCarlo
Configuration cubic_side_length 8 particle_type0 /home/user/feasst/particle/lj.txt  
Potential Model LennardJones  
Potential VisitModel LongRangeCorrections  
ThermoParams beta 0.6666666666666666 chemical_potential0 -2.352321  
FlatHistogram Bias TransitionMatrix Macrostate MacrostateNumParticles max 5 min 0 min_sweeps 50 width 1  
TrialTranslate tunable_param 1 weight 0.25  
TrialTransfer particle_type 0 weight 1  
Log output_file lj.txt trials_per_write 1e5  
Movie output_file lj.xyz trials_per_write 1e5  
CheckEnergy tolerance 1e-8 trials_per_update 1e5  
Tune  
CriteriaUpdater trials_per_update 1e5  
CriteriaWriter output_file lj_fh.txt trials_per_write 1e5  
Energy multistate true output_file lj_en.txt trials_per_update 1 trials_per_write 1e5  
Checkpoint checkpoint_file checkpoint.fst num_hours 0.0001  
Run until complete  
# initializing random number generator with seed: 1744120344
 
 exit: 0


In [3]:
import pandas as pd

"""Compare the free energies and potential energies with the NIST SRSW
https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website
https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm
"""
fh2=pd.read_csv('lj_fh.txt', comment='#')
en=pd.read_csv('lj_en.txt')
print(fh2[['delta_ln_prob', 'delta_ln_prob_stdev']])
print('energy', en[['average', 'block_stdev']])
for macro in range(1, len(fh2)):
    assert np.abs(fh["delta_ln_prob"][macro] - fh2["delta_ln_prob"][macro]) < 4*fh2["delta_ln_prob_stdev"][macro]
    assert np.abs(fh['energy'][macro] - en['average'][macro]) < 4*(fh["energystd"][macro]**2 + en['block_stdev'][macro]**2)**(1./2.)

   delta_ln_prob  delta_ln_prob_stdev
0            NaN             0.000000
1        4.67080             0.001936
2        3.98881             0.001782
3        3.59203             0.001857
4        3.31280             0.001779
5        3.09921             0.002264
energy         average   block_stdev
0  7.401831e-17  3.657879e-16
1 -6.057400e-04  7.702370e-10
2 -3.077346e-02  1.917334e-04
3 -8.983129e-02  5.428819e-04
4 -1.793143e-01  6.927072e-04
5 -2.975640e-01  1.246197e-03


A number of files should also have been created.
If the [FlatHistogram](../../flat_histogram/doc/FlatHistogram_arguments.rst) method is sampling perfectly, the simulation performs a random walk along the [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst).
For larger ranges of [Macrostates](../../flat_histogram/doc/Macrostate_arguments.rst), or for more difficult sampling cases, monitoring the [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst) can help you determine what conditions are preventing convergence.
For example, the [Macrostate](../../flat_histogram/doc/Macrostate_arguments.rst) as a function of the number of attempts may look like the following:

In [5]:
pd.read_csv("lj.txt", header=0).dropna(axis='columns')

,volume,num_particles_of_type0,beta,state,energy,LennardJones,LongRangeCorrections,BondTwoBody,BondThreeBody,BondFourBody,trial,TrialTranslate,tunable,TrialAdd,TrialRemove
0,512,5,0.666667,5,-0.0358926,-0.0207491,-0.0151435,0,0,0,100000,0.927692,2.86117,0.0434881,0.0431646
1,512,2,0.666667,2,-0.00242296,1.33227e-15,-0.00242296,0,0,0,200000,0.945359,2.86117,0.374312,0.397731
2,512,2,0.666667,2,-0.00242296,-1.94636e-15,-0.00242296,0,0,0,300000,0.950461,2.86117,0.519903,0.574258
3,512,4,0.666667,4,-0.627471,-0.617779,-0.00969184,0,0,0,400000,0.952717,2.86117,0.593767,0.668579
4,512,5,0.666667,5,-0.0409858,-0.0258423,-0.0151435,0,0,0,500000,0.954323,2.86117,0.637489,0.726514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,512,1,0.666667,1,-0.00060574,-5.54765e-15,-0.00060574,0,0,0,4800000,0.958891,2.81568,0.787615,0.93856
101,512,2,0.666667,2,-0.00242296,-1.07553e-15,-0.00242296,0,0,0,4900000,0.958986,2.81568,0.78809,0.939261
102,512,4,0.666667,4,-0.0742168,-0.0645249,-0.00969184,0,0,0,5000000,0.958993,2.81568,0.788526,0.939941
103,512,3,0.666667,3,-0.112772,-0.10732,-0.00545166,0,0,0,5100000,0.959008,2.81568,0.788991,0.940624


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!